In [1]:
from sklearn import cross_validation, datasets
import numpy as np

In [2]:
boston = datasets.load_boston()

In [3]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(boston.data, boston.target, 
                                                                                     test_size = 0.25)

In [4]:
class Node:
    def __init__(self, best_feature, best_threshold, parent = 0, left_child = 0, right_child = 0):
        self.best_feature = best_feature
        self.best_threshold = best_threshold
        self.parent = parent
        self.mean = 0
        self.depth = 0
        self.left_child = left_child
        self.right_child = right_child

In [5]:
def H(target):
    mean = np.mean(target)
    if (len(target) != 0):
        return (1 / len(target)) * np.sum((target-mean)**2)
    else:
        return 1
#     return 2 * (np.sum(data) / len(data)) * (1 - np.sum(data) / len(data))

In [6]:
def split(data, target, num_feature, threshold):
    data_right = []
    data_left = []
    target_right = []
    target_left = []

    for i in range(len(data)):     
        a = data[i]
        
        print(type(a))
        len(a)
        
        if(data[i][num_feature] < threshold):
            data_left.append(data[i][num_feature])
            target_left.append(target[i])
        else:
            data_right.append(data[i][num_feature])
            target_right.append(target[i])
            
    return data_left, target_left, data_right, target_right

In [7]:
def G(i, t, data, target):
    data_left, target_left, data_right, target_right = split(data, target, i, t)
    L = len(data_left)
    R = len(data_right)
    Q = L + R
    G = L / Q * H(target_left) + R / Q * H(target_right)
    return G

In [8]:
def func(data, target):
    min_G = 10000
    best_feature = 0
    best_thresholds = 0        
    for cur_feature in range(13):
        max_feat = np.max(X_train.T[cur_feature])
        min_feat = np.min(X_train.T[cur_feature])
        thresholds = np.linspace(min_feat, max_feat, 100)
        for thresh in thresholds:
            if G(cur_feature, thresh, data, target) < min_G:
                min_G = G(cur_feature, thresh, data, target)
                best_feature = cur_feature
                best_thresholds = thresh
    return best_feature, best_thresholds

In [9]:
def build_tree(node, X_train, y_train, depth, num_in_node):
    best_feature, best_thresholds = func(X_train, y_train)
    
    node.best_feature = best_feature
    node.best_thresholds = best_thresholds
    
    left_child_data, left_child_target, right_child_data, right_child_target = split(X_train, y_train, best_feature,
                                                                                                        best_thresholds)
    left_child = Node(0, 0, node)
    right_child = Node(0, 0, node)
    
    left_child.mean = np.mean(left_child_target)
    right_child.mean = np.mean(right_child_target)

    left_child.depth = left_child.parent.depth + 1
    right_child.depth = right_child.parent.depth + 1
    
    node.left_child = left_child
    node.right_child = right_child
    
    if (len(left_child_data) > num_in_node and node.depth <= depth):
        build_tree(node.left_child, left_child_data, left_child_target, depth, num_in_node)
        
    if (len(right_child_data) > num_in_node and node.depth <= depth):
        build_tree(node.right_child, right_child_data, right_child_target, depth, num_in_node)

In [10]:
class DecisionTree:
    def __init__(self):
        self._node = Node(0, 0)
    def fit(self, X_train, y_train, max_depth = 5, num_in_node = 5):
        build_tree(self._node, X_train, y_train, max_depth, num_in_node)
    def predict(self, X_test):
        y = []
        for x in X_test:
            while (self.node.left_child != 0 or self.node.right_child != 0):
                if (x[self.node.best_feature] < self.node.best_thresold ):
                    self.node = self.node.left_child
                else:
                    self.node = self.node.right_child
            y.append(self.node.mean)
        return np.array(y)
                

# test

In [11]:
myTree = DecisionTree()
myTree.fit(X_train, y_train)

<class 'numpy.ndarray'>


UnboundLocalError: local variable 'b' referenced before assignment

In [ ]:
y_predicted = myTree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
